# Batch tính độ mặn (salinity) cho các raster Landsat 2022
Xử lý tất cả file .tif trong `data/processed/landsat_salinity_2022_full_year` với H3 grid đã làm sạch.
- Tạo H3 grid (res 7) với chiến lược Buffer → Polyfill → Intersect
- Loại bỏ đảo nhỏ <600 km² từ ranh giới ĐBSCL
- Tính salinity features: min/max/mean/std + % pixel ≥ 0.2/0.5/1.0 ppt + phân cấp rủi ro
- Đầu ra: `salinity_h3_features_2022.csv` trong cùng thư mục

In [14]:
import os
from pathlib import Path
import glob
import numpy as np
import pandas as pd
import geopandas as gpd
import rasterio
from rasterio.mask import mask as rasterio_mask
from shapely.geometry import Polygon, MultiPolygon
from shapely.ops import unary_union
import h3

# Xác định project root
PROJECT_ROOT = Path.cwd()
if not (PROJECT_ROOT / 'data').exists() and (PROJECT_ROOT.parent / 'data').exists():
    PROJECT_ROOT = PROJECT_ROOT.parent

SALINITY_DIR = PROJECT_ROOT / 'data' / 'processed' / 'landsat_salinity_2022_full_year'
BOUNDARY_PATH = PROJECT_ROOT / 'data' / 'raw' / 'DBSCL_Boundary_Clean.shp'
H3_GRID_OUT = SALINITY_DIR / 'h3_grid_dbscl_clean.geojson'
OUTPUT_CSV = SALINITY_DIR / 'salinity_h3_features_2022.csv'

SALINITY_DIR.mkdir(parents=True, exist_ok=True)

print(f'✅ Project root: {PROJECT_ROOT}')
print(f'✅ Thư mục salinity: {SALINITY_DIR}')
print(f'✅ Boundary: {BOUNDARY_PATH}')
print(f'✅ Output: {OUTPUT_CSV}')

✅ Project root: d:\Mekong_DGGS
✅ Thư mục salinity: d:\Mekong_DGGS\data\processed\landsat_salinity_2022_full_year
✅ Boundary: d:\Mekong_DGGS\data\raw\DBSCL_Boundary_Clean.shp
✅ Output: d:\Mekong_DGGS\data\processed\landsat_salinity_2022_full_year\salinity_h3_features_2022.csv


In [15]:
# Tạo H3 grid sạch (loại đảo <600 km²) với chiến lược Buffer → Polyfill → Intersect
RES = 7
MIN_AREA_KM2 = 600
H3_EDGE_LEN_METERS = {5: 8544.4, 6: 3229.6, 7: 1220.6, 8: 461.4, 9: 174.4, 10: 65.9}
BUFFER_DIST = H3_EDGE_LEN_METERS.get(RES, 2000) * 2.0

def clean_roi(roi_gdf, min_area_km2=600):
    """Loại bỏ đảo nhỏ"""
    gdf_metric = roi_gdf.to_crs("EPSG:32648")
    gdf_exploded = gdf_metric.explode(index_parts=True).reset_index(drop=True)
    gdf_exploded["area_km2"] = gdf_exploded.geometry.area / 1e6
    gdf_clean = gdf_exploded[gdf_exploded["area_km2"] > min_area_km2].copy()
    gdf_dissolved = gdf_clean.dissolve()
    return gdf_dissolved.to_crs("EPSG:4326")

def get_h3_funcs():
    v = int(h3.__version__.split('.')[0])
    return (h3.latlng_to_cell, h3.cell_to_boundary) if v >= 4 else (h3.geo_to_h3, h3.h3_to_geo_boundary)

to_cell_func, to_boundary_func = get_h3_funcs()

def generate_h3_grid(gdf, resolution):
    """Buffer → Polyfill → Intersect"""
    gdf_metric = gdf.to_crs("EPSG:32648")
    gdf_buffered = gdf_metric.copy()
    gdf_buffered["geometry"] = gdf_metric.buffer(BUFFER_DIST)
    gdf_buffered_ll = gdf_buffered.to_crs("EPSG:4326")
    
    hex_set = set()
    for _, row in gdf_buffered_ll.iterrows():
        geom = row.geometry
        geoms_list = geom.geoms if hasattr(geom, "geoms") else [geom]
        for single_geom in geoms_list:
            try:
                if hasattr(h3, "polygon_to_cells"):
                    from h3 import LatLngPoly
                    outer = [(lat, lon) for lon, lat in single_geom.exterior.coords]
                    holes = [[(lat, lon) for lon, lat in interior.coords] for interior in single_geom.interiors]
                    hexes = h3.polygon_to_cells(LatLngPoly(outer, holes), resolution)
                else:
                    from shapely.geometry import mapping
                    hexes = h3.polyfill(mapping(single_geom), resolution, geo_json_conformant=True)
                hex_set.update(hexes)
            except Exception as e:
                print(f"⚠️ Polyfill: {e}")
    
    union_poly = unary_union(gdf.geometry.values)
    valid_hexes, hex_polys = [], []
    for h_idx in hex_set:
        try:
            boundary = to_boundary_func(h_idx)
            poly_coords = [(lon, lat) for lat, lon in boundary] if hasattr(h3, "cell_to_boundary") else boundary
            poly_geom = Polygon(poly_coords)
            if poly_geom.intersects(union_poly):
                valid_hexes.append(h_idx)
                hex_polys.append(poly_geom)
        except:
            pass
    
    return gpd.GeoDataFrame({"h3_index": valid_hexes}, geometry=hex_polys, crs="EPSG:4326")

# Load boundary và tạo H3
roi_raw = gpd.read_file(BOUNDARY_PATH)
roi_clean = clean_roi(roi_raw, MIN_AREA_KM2)
print(f"✅ ROI sạch: {len(roi_clean)} feature")

h3_grid = generate_h3_grid(roi_clean, RES)
print(f"✅ H3 grid (res {RES}): {len(h3_grid)} ô")

h3_grid.to_file(H3_GRID_OUT, driver="GeoJSON")
print(f"✅ Lưu: {H3_GRID_OUT}")

✅ ROI sạch: 1 feature
✅ H3 grid (res 7): 6959 ô
✅ Lưu: d:\Mekong_DGGS\data\processed\landsat_salinity_2022_full_year\h3_grid_dbscl_clean.geojson


In [16]:
def process_salinity(gdf, tiff_path):
    """Tính salinity features cho từng ô H3"""
    results = []
    with rasterio.open(tiff_path) as src:
        gdf_crs = gdf.to_crs(src.crs)
        for geom, h3_idx in zip(gdf_crs.geometry, gdf['h3_index']):
            row = {
                'h3_index': h3_idx,
                'salinity_min': np.nan,
                'salinity_max': np.nan,
                'salinity_mean': np.nan,
                'salinity_std': np.nan,
                'pct_salinity_pixels': 0.0,
                'pct_salinity_gte_0_2': 0.0,
                'pct_salinity_gte_0_5': 0.0,
                'pct_salinity_gte_1_0': 0.0
            }
            try:
                masked, _ = rasterio_mask(src, [geom], crop=True)
                data = masked[0].astype(float).flatten()
                data_valid = data[(data >= -100) & (data <= 100) & ~np.isnan(data)]
                total_pixels = data.size
                
                if data_valid.size > 0:
                    row['salinity_min'] = float(np.nanmin(data_valid))
                    row['salinity_max'] = float(np.nanmax(data_valid))
                    row['salinity_mean'] = float(np.nanmean(data_valid))
                    row['salinity_std'] = float(np.nanstd(data_valid))
                    row['pct_salinity_pixels'] = (data_valid.size / total_pixels * 100) if total_pixels > 0 else 0.0
                    row['pct_salinity_gte_0_2'] = (np.sum(data_valid >= 0.2) / data_valid.size * 100)
                    row['pct_salinity_gte_0_5'] = (np.sum(data_valid >= 0.5) / data_valid.size * 100)
                    row['pct_salinity_gte_1_0'] = (np.sum(data_valid >= 1.0) / data_valid.size * 100)
            except:
                pass
            results.append(row)
    return pd.DataFrame(results)

print("✅ Hàm xử lý salinity đã sẵn sàng")

✅ Hàm xử lý salinity đã sẵn sàng


In [17]:
# Liệt kê tất cả file salinity .tif
tif_files = sorted(glob.glob(str(SALINITY_DIR / '*.tif')))
print(f'📁 Tìm thấy {len(tif_files)} file raster')
for f in tif_files[:5]:
    print(f'  - {os.path.basename(f)}')
if len(tif_files) > 5:
    print(f'  ... và {len(tif_files) - 5} file khác')

📁 Tìm thấy 12 file raster
  - 2022_M01_L89_Salinity.tif
  - 2022_M02_L89_Salinity.tif
  - 2022_M03_L89_Salinity.tif
  - 2022_M04_L89_Salinity.tif
  - 2022_M05_L89_Salinity.tif
  ... và 7 file khác


In [18]:
# Kiểm tra và sửa file corrupt (nếu có)
def validate_and_fix_tif(tif_path):
    """Kiểm tra file .tif, thử fix nếu corrupt"""
    filename = os.path.basename(tif_path)
    
    # Test 1: Thử mở file basic
    try:
        with rasterio.open(tif_path) as src:
            _ = src.crs
            _ = src.bounds
        return True, "OK"
    except Exception as e:
        error_msg = str(e)
        
        # Test 2: Thử với GDAL options khác
        try:
            with rasterio.Env(GDAL_DISABLE_READDIR_ON_OPEN='EMPTY_DIR'):
                with rasterio.open(tif_path, OVERVIEW_LEVEL=0) as src:
                    _ = src.crs
            return True, "Fixed with GDAL options"
        except:
            pass
        
        return False, error_msg[:100]

print("🔍 Kiểm tra tất cả file .tif...")
valid_files = []
corrupt_files = []

for tif_path in tif_files:
    filename = os.path.basename(tif_path)
    is_valid, msg = validate_and_fix_tif(tif_path)
    if is_valid:
        valid_files.append(tif_path)
        if msg != "OK":
            print(f"  ⚠️  {filename}: {msg}")
    else:
        corrupt_files.append((filename, msg))
        print(f"  ❌ {filename}: CORRUPT - {msg[:60]}")

print(f"\n✅ File hợp lệ: {len(valid_files)}/{len(tif_files)}")
if corrupt_files:
    print(f"❌ File corrupt: {len(corrupt_files)}")
    print("\n💡 Khuyến nghị: Tải lại các file corrupt từ nguồn hoặc xóa khỏi thư mục")

# Cập nhật danh sách file để xử lý
tif_files = valid_files

🔍 Kiểm tra tất cả file .tif...

✅ File hợp lệ: 12/12


In [19]:
# Phân tích chi tiết file corrupt
if corrupt_files:
    print("📊 PHÂN TÍCH FILE CORRUPT:\n")
    
    # Lấy kích thước file hợp lệ để so sánh
    valid_sizes = [os.path.getsize(f) for f in valid_files]
    avg_size = np.mean(valid_sizes) if valid_sizes else 0
    min_size = np.min(valid_sizes) if valid_sizes else 0
    max_size = np.max(valid_sizes) if valid_sizes else 0
    
    print(f"Kích thước file hợp lệ:")
    print(f"  Trung bình: {avg_size/1024/1024:.2f} MB")
    print(f"  Min-Max: {min_size/1024/1024:.2f} - {max_size/1024/1024:.2f} MB\n")
    
    for corrupt_file, error in corrupt_files:
        corrupt_path = SALINITY_DIR / corrupt_file
        if corrupt_path.exists():
            corrupt_size = os.path.getsize(corrupt_path)
            size_ratio = (corrupt_size / avg_size * 100) if avg_size > 0 else 0
            
            print(f"❌ {corrupt_file}:")
            print(f"   Kích thước: {corrupt_size/1024/1024:.2f} MB ({size_ratio:.1f}% so với TB)")
            print(f"   Lỗi: {error[:100]}")
            
            if size_ratio < 50:
                print(f"   ⚠️  FILE CHƯA TẢI XONG (chỉ {size_ratio:.1f}%) - Tải lại từ nguồn")
            elif size_ratio < 90:
                print(f"   ⚠️  FILE BỊ THIẾU DỮ LIỆU ({size_ratio:.1f}%) - Tải lại hoặc export lại")
            else:
                print(f"   ⚠️  FILE GẦN ĐỦ ({size_ratio:.1f}%) nhưng header/metadata bị lỗi")
            
            print(f"   💡 Giải pháp:")
            print(f"      1. Xóa: del {corrupt_path}")
            print(f"      2. Tải lại từ Google Drive/GEE")
            print(f"      3. Hoặc bỏ qua (đã skip tự động)\n")
else:
    print("✅ Tất cả file đều hợp lệ!")

✅ Tất cả file đều hợp lệ!


In [20]:
# Xử lý batch tất cả file và gộp kết quả
all_results = []
failed_files = []

for i, tif_path in enumerate(tif_files, 1):
    filename = os.path.basename(tif_path)
    print(f'🔄 [{i}/{len(tif_files)}] {filename}')
    try:
        df = process_salinity(h3_grid, tif_path)
        df['source_file'] = filename
        all_results.append(df)
        print(f'   ✅ {len(df)} ô xử lý xong')
    except Exception as e:
        print(f'   ❌ Lỗi: {str(e)[:80]}...')
        failed_files.append((filename, str(e)))

if all_results:
    merged = pd.concat(all_results, ignore_index=True)
    
    # Tạo cột time từ source_file (VD: 2022_M01_L89_Salinity.tif -> 01)
    import re
    merged['time'] = merged['source_file'].str.extract(r'_M(\d{2})_', expand=False)
    
    # Bỏ cột source_file
    merged = merged.drop(columns=['source_file'])
    
    # Sắp xếp cột: h3_index, time, các cột salinity
    cols = ['h3_index', 'time'] + [c for c in merged.columns if c not in ['h3_index', 'time']]
    merged = merged[cols]
    
    # Lưu CSV
    merged.to_csv(OUTPUT_CSV, index=False)
    print(f'\n✅ Lưu: {OUTPUT_CSV}')
    print(f'📊 Tổng: {len(merged)} dòng từ {len(all_results)}/{len(tif_files)} file')
    
    if failed_files:
        print(f'\n⚠️  {len(failed_files)} file bị lỗi:')
        for fname, err in failed_files:
            print(f'   - {fname}')
    
    # Thống kê nhanh
    print('\n📊 Phân bố theo tháng:')
    for month, count in merged['time'].value_counts().sort_index().items():
        print(f'   Tháng {month}: {count} dòng')
    
    print('\n📊 Preview 5 dòng:')
    preview = merged[['h3_index', 'time', 'salinity_mean', 'salinity_std', 
                      'pct_salinity_gte_0_2', 'pct_salinity_gte_1_0']].head()
    print(preview.to_string())
else:
    print('❌ Không có file .tif nào xử lý thành công')

🔄 [1/12] 2022_M01_L89_Salinity.tif
   ✅ 6959 ô xử lý xong
🔄 [2/12] 2022_M02_L89_Salinity.tif
   ✅ 6959 ô xử lý xong
🔄 [3/12] 2022_M03_L89_Salinity.tif
   ✅ 6959 ô xử lý xong
🔄 [4/12] 2022_M04_L89_Salinity.tif
   ✅ 6959 ô xử lý xong
🔄 [5/12] 2022_M05_L89_Salinity.tif
   ✅ 6959 ô xử lý xong
🔄 [6/12] 2022_M06_L89_Salinity.tif
   ✅ 6959 ô xử lý xong
🔄 [7/12] 2022_M07_L89_Salinity.tif
   ✅ 6959 ô xử lý xong
🔄 [8/12] 2022_M08_L89_Salinity.tif
   ✅ 6959 ô xử lý xong
🔄 [9/12] 2022_M09_L89_Salinity.tif
   ✅ 6959 ô xử lý xong
🔄 [10/12] 2022_M10_L89_Salinity.tif
   ✅ 6959 ô xử lý xong
🔄 [11/12] 2022_M11_L89_Salinity.tif
   ✅ 6959 ô xử lý xong
🔄 [12/12] 2022_M12_L89_Salinity.tif
   ✅ 6959 ô xử lý xong

✅ Lưu: d:\Mekong_DGGS\data\processed\landsat_salinity_2022_full_year\salinity_h3_features_2022.csv
📊 Tổng: 83508 dòng từ 12/12 file

📊 Phân bố theo tháng:
   Tháng 01: 6959 dòng
   Tháng 02: 6959 dòng
   Tháng 03: 6959 dòng
   Tháng 04: 6959 dòng
   Tháng 05: 6959 dòng
   Tháng 06: 6959 dòng
   Thán